In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!git clone https://github.com/TeamNops/SIH_UI_Geoserver

Cloning into 'SIH_UI_Geoserver'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (156/156), done.
remote: Total 164 (delta 8), reused 163 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (164/164), 31.24 MiB | 48.40 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [9]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
from typing import Generator, Tuple, List
from glob import glob

class FILMDataGenerator:
    def __init__(self, 
                 data_dir: str,
                 batch_size: int = 8,
                 image_size: Tuple[int, int] = (256, 256)):
        """
        Initialize the data generator for FILM model training
        
        Args:
            data_dir: Root directory containing sample folders
            batch_size: Number of samples per batch
            image_size: Target size for the images (height, width)
        """
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.image_size = image_size
        
        # Get all sample directories
        self.sample_dirs = [d for d in glob(os.path.join(data_dir, "sample_*")) 
                          if os.path.isdir(d)]
        print(f"Found {len(self.sample_dirs)} sample directories")

    def load_and_preprocess_image(self, image_path: str) -> tf.Tensor:
        """Load and preprocess a single image"""
        # Read image file
        image = tf.io.read_file(image_path)
        # Decode image
        image = tf.io.decode_image(image, channels=3)
        # Convert to float32 and normalize to [0, 1]
        image = tf.cast(image, tf.float32) / 255.0
        # Resize image
        image = tf.image.resize(image, self.image_size)
        return image

    def create_dataset(self) -> tf.data.Dataset:
        """Create a TensorFlow dataset for training"""
        def generator():
            while True:
                # Randomly sample batch_size directories
                batch_dirs = np.random.choice(self.sample_dirs, 
                                           size=self.batch_size)
                
                times = []
                images_0 = []
                images_1 = []
                images_mid = []  # Ground truth middle frames
                
                for dir_path in batch_dirs:
                    # Load the three images
                    img0 = self.load_and_preprocess_image(
                        os.path.join(dir_path, "image.png"))
                    img1 = self.load_and_preprocess_image(
                        os.path.join(dir_path, "image1.png"))
                    img2 = self.load_and_preprocess_image(
                        os.path.join(dir_path, "image2.png"))
                    
                    times.append([0.5])  # Middle frame time
                    images_0.append(img0)
                    images_1.append(img2)
                    images_mid.append(img1)
                
                # Stack into batches
                yield {
                    'time': tf.convert_to_tensor(times, dtype=tf.float32),
                    'x0': tf.stack(images_0),
                    'x1': tf.stack(images_1)
                }, tf.stack(images_mid)

        # Create dataset from generator
        dataset = tf.data.Dataset.from_generator(
            generator,
            output_signature=(
                {
                    'time': tf.TensorSpec(shape=(None, 1), dtype=tf.float32),
                    'x0': tf.TensorSpec(shape=(None, *self.image_size, 3), 
                                      dtype=tf.float32),
                    'x1': tf.TensorSpec(shape=(None, *self.image_size, 3), 
                                      dtype=tf.float32)
                },
                tf.TensorSpec(shape=(None, *self.image_size, 3), 
                             dtype=tf.float32)
            )
        )
        
        return dataset.prefetch(tf.data.AUTOTUNE)

class FILMFineTuner:
    def __init__(self,
                 base_model_url: str = "https://tfhub.dev/google/film/1",
                 learning_rate: float = 1e-4):
        """
        Initialize FILM model fine-tuner
        
        Args:
            base_model_url: URL for the base FILM model
            learning_rate: Learning rate for training
        """
        self.model = hub.load(base_model_url)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        
        # Define loss function (L1 loss)
        self.loss_fn = tf.keras.losses.MeanAbsoluteError()
        
    @tf.function
    def train_step(self, 
                  inputs: dict,
                  target: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
        """
        Print shapes for debugging
        """
        tf.print("Input shapes:", 
                "time:", inputs['time'].shape,
                "x0:", inputs['x0'].shape,
                "x1:", inputs['x1'].shape,
                "target:", target.shape)
        """
        Perform one training step
        
        Args:
            inputs: Dictionary containing 'time', 'x0', and 'x1'
            target: Ground truth middle frame
            
        Returns:
            Tuple of (loss value, predicted frame)
        """
        with tf.GradientTape() as tape:
            # Forward pass
            output_dict = self.model(inputs)
            # The model returns a dictionary - extract the interpolated frame
            predicted = output_dict['image']  # Get the interpolated frame from output
            
            # Calculate loss
            loss = self.loss_fn(target, predicted)
            
        # Calculate gradients
        gradients = tape.gradient(loss, self.model.trainable_variables)
        
        # Apply gradients
        self.optimizer.apply_gradients(
            zip(gradients, self.model.trainable_variables))
        
        return loss, predicted

def main():
    # Check if dataset directory exists
    if not os.path.exists("/kaggle/working/SIH_UI_Geoserver/Frame_Interpolation/interpolation_dataset"):
        raise FileNotFoundError("interpolation_dataset directory not found")
        
    # Training parameters
    BATCH_SIZE = 8
    IMAGE_SIZE = (256, 256)
    EPOCHS = 10
    
    # Initialize data generator
    data_generator = FILMDataGenerator(
        data_dir="/kaggle/working/SIH_UI_Geoserver/Frame_Interpolation/interpolation_dataset",
        batch_size=BATCH_SIZE,
        image_size=IMAGE_SIZE
    )
    
    # Create dataset
    dataset = data_generator.create_dataset()
    
    # Initialize model
    trainer = FILMFineTuner()
    
    # Training loop
    for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch + 1}/{EPOCHS}")
        
        # Initialize metrics
        epoch_loss = tf.keras.metrics.Mean()
        
        for step, (inputs, target) in enumerate(dataset):
            # Perform training step
            loss, predicted = trainer.train_step(inputs, target)
            
            # Update metrics
            epoch_loss.update_state(loss)
            
            if step % 10 == 0:
                print(f"Step {step} - Loss: {epoch_loss.result():.4f}")
            
            # Optional: save checkpoints periodically
            if step % 1000 == 0:
                # Save model weights
                checkpoint_dir = f"checkpoints/epoch_{epoch}_step_{step}"
                os.makedirs(checkpoint_dir, exist_ok=True)
                 

if __name__ == "__main__":
    main()

Found 43 sample directories

Epoch 1/10



KeyboardInterrupt



In [10]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
from typing import Generator, Tuple, List
from glob import glob

class FILMDataGenerator:
    def _init_(self, 
                 data_dir: str,
                 batch_size: int = 8,
                 image_size: Tuple[int, int] = (256, 256)):
        """
        Initialize the data generator for FILM model training
        
        Args:
            data_dir: Root directory containing sample folders
            batch_size: Number of samples per batch
            image_size: Target size for the images (height, width)
        """
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.image_size = image_size
        
        # Get all sample directories
        self.sample_dirs = [d for d in glob(os.path.join(data_dir, "sample_*")) 
                            if os.path.isdir(d)]
        print(f"Found {len(self.sample_dirs)} sample directories")

    def load_and_preprocess_image(self, image_path: str) -> tf.Tensor:
        """Load and preprocess a single image"""
        # Read image file
        image = tf.io.read_file(image_path)
        # Decode image
        image = tf.io.decode_image(image, channels=3)
        # Convert to float32 and normalize to [0, 1]
        image = tf.cast(image, tf.float32) / 255.0
        # Resize image
        image = tf.image.resize(image, self.image_size)
        return image

    def create_dataset(self) -> tf.data.Dataset:
        """Create a TensorFlow dataset for training"""
        def generator():
            while True:
                # Randomly sample batch_size directories
                batch_dirs = np.random.choice(self.sample_dirs, 
                                              size=self.batch_size)
                
                times = []
                images_0 = []
                images_1 = []
                images_mid = []  # Ground truth middle frames
                
                for dir_path in batch_dirs:
                    # Load the three images
                    img0 = self.load_and_preprocess_image(
                        os.path.join(dir_path, "image.png"))
                    img1 = self.load_and_preprocess_image(
                        os.path.join(dir_path, "image1.png"))
                    img2 = self.load_and_preprocess_image(
                        os.path.join(dir_path, "image2.png"))
                    
                    times.append([0.5])  # Middle frame time
                    images_0.append(img0)
                    images_1.append(img2)
                    images_mid.append(img1)
                
                # Stack into batches
                yield {
                    'time': tf.convert_to_tensor(times, dtype=tf.float32),
                    'x0': tf.stack(images_0),
                    'x1': tf.stack(images_1)
                }, tf.stack(images_mid)

        # Create dataset from generator
        dataset = tf.data.Dataset.from_generator(
            generator,
            output_signature=(
                {
                    'time': tf.TensorSpec(shape=(None, 1), dtype=tf.float32),
                    'x0': tf.TensorSpec(shape=(None, *self.image_size, 3), dtype=tf.float32),
                    'x1': tf.TensorSpec(shape=(None, *self.image_size, 3), dtype=tf.float32)
                },
                tf.TensorSpec(shape=(None, *self.image_size, 3), dtype=tf.float32)
            )
        )
        
        # Prefetch for performance
        dataset = dataset.prefetch(tf.data.AUTOTUNE)
        return dataset

class FILMFineTuner:
    def _init_(self,
                 base_model_url: str = "https://tfhub.dev/google/film/1",
                 learning_rate: float = 1e-4):
        """
        Initialize FILM model fine-tuner
        
        Args:
            base_model_url: URL for the base FILM model
            learning_rate: Learning rate for training
        """
        self.model = hub.load(base_model_url)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        
        # Define loss function (L1 loss)
        self.loss_fn = tf.keras.losses.MeanAbsoluteError()
        
    @tf.function
    def train_step(self, inputs: dict, target: tf.Tensor):
        """
        Perform one training step
        
        Args:
            inputs: Dictionary containing 'time', 'x0', and 'x1'
            target: Ground truth middle frame
            
        Returns:
            loss value, predicted frame
        """
        with tf.GradientTape() as tape:
            output_dict = self.model(inputs)
            predicted = output_dict['image']  # interpolated frame
            loss = self.loss_fn(target, predicted)

        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(
            zip(gradients, self.model.trainable_variables))
        
        return loss, predicted

def main():
    # Check if dataset directory exists
    data_dir = "/kaggle/working/SIH_UI_Geoserver/Frame_Interpolation/interpolation_dataset"
    if not os.path.exists(data_dir):
        raise FileNotFoundError("interpolation_dataset directory not found")
        
    # Training parameters
    BATCH_SIZE = 8
    IMAGE_SIZE = (256, 256)
    EPOCHS = 10

    # Create the MirroredStrategy for multi-GPU training
    strategy = tf.distribute.MirroredStrategy()
    print("Number of devices: ", strategy.num_replicas_in_sync)

    # Initialize data generator
    data_generator = FILMDataGenerator(
        data_dir=data_dir,
        batch_size=BATCH_SIZE,
        image_size=IMAGE_SIZE
    )
    
    # Create dataset
    dataset = data_generator.create_dataset()
    
    # Distribute the dataset
    # Each batch returned will be split across the available GPUs.
    dataset = strategy.experimental_distribute_dataset(dataset)
    
    with strategy.scope():
        # Initialize model (done under strategy scope)
        trainer = FILMFineTuner()

        # Custom training loop across epochs
        for epoch in range(EPOCHS):
            print(f"\nEpoch {epoch + 1}/{EPOCHS}")
            
            epoch_loss = tf.keras.metrics.Mean()

            # Define a distributed train step to run on each replica
            @tf.function
            def distributed_train_step(dist_inputs, dist_target):
                per_replica_losses, _ = strategy.run(
                    trainer.train_step, 
                    args=(dist_inputs, dist_target)
                )
                # per_replica_losses is a per-replica tensor, need to reduce it
                mean_loss = strategy.reduce(tf.distribute.ReduceOp.MEAN, per_replica_losses, axis=None)
                return mean_loss
            
            for step, (inputs, target) in enumerate(dataset):
                # One train step
                loss = distributed_train_step(inputs, target)
                epoch_loss.update_state(loss)

                if step % 10 == 0:
                    print(f"Step {step} - Loss: {epoch_loss.result():.4f}")
                
                # Optional checkpoint saving (not fully implemented)
                if step % 1000 == 0 and step > 0:
                    checkpoint_dir = f"checkpoints/epoch_{epoch}_step{step}"
                    os.makedirs(checkpoint_dir, exist_ok=True)
                    # Here you would implement saving logic for model variables if desired.

if __name__ == "__main__":
    main()

Number of devices:  2


TypeError: FILMDataGenerator() takes no arguments

In [15]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
from typing import Generator, Tuple, List
from glob import glob
class FILMDataGenerator:
    def __init__(self, 
                 data_dir: str,
                 batch_size: int = 8,
                 image_size: Tuple[int, int] = (256, 256)):
        """
        Initialize the data generator for FILM model training
        
        Args:
            data_dir: Root directory containing sample folders
            batch_size: Number of samples per batch
            image_size: Target size for the images (height, width)
        """
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.image_size = image_size
        
        # Get all sample directories
        self.sample_dirs = [d for d in glob(os.path.join(data_dir, "sample_*")) 
                            if os.path.isdir(d)]
        print(f"Found {len(self.sample_dirs)} sample directories")
        
        # Calculate steps per epoch
        self.steps_per_epoch = len(self.sample_dirs) // self.batch_size
        if len(self.sample_dirs) % self.batch_size != 0:
            self.steps_per_epoch += 1
        print(f"Steps per epoch: {self.steps_per_epoch}")

    def load_and_preprocess_image(self, image_path: str) -> tf.Tensor:
        """Load and preprocess a single image"""
        # Read image file
        image = tf.io.read_file(image_path)
        # Decode image
        image = tf.io.decode_image(image, channels=3)
        # Convert to float32 and normalize to [0, 1]
        image = tf.cast(image, tf.float32) / 255.0
        # Resize image
        image = tf.image.resize(image, self.image_size)
        return image

    def create_dataset(self) -> tf.data.Dataset:
        """Create a TensorFlow dataset for training"""
        def generator():
            # Shuffle the sample directories
            indices = np.arange(len(self.sample_dirs))
            np.random.shuffle(indices)
            
            # Process in batches
            for i in range(0, len(indices), self.batch_size):
                batch_indices = indices[i:i + self.batch_size]
                batch_dirs = [self.sample_dirs[idx] for idx in batch_indices]
                
                times = []
                images_0 = []
                images_1 = []
                images_mid = []
                
                for dir_path in batch_dirs:
                    # Load the three images
                    img0 = self.load_and_preprocess_image(
                        os.path.join(dir_path, "image.png"))
                    img1 = self.load_and_preprocess_image(
                        os.path.join(dir_path, "image1.png"))
                    img2 = self.load_and_preprocess_image(
                        os.path.join(dir_path, "image2.png"))
                    
                    times.append([0.5])
                    images_0.append(img0)
                    images_1.append(img2)
                    images_mid.append(img1)
                
                # Stack into batches
                yield {
                    'time': tf.convert_to_tensor(times, dtype=tf.float32),
                    'x0': tf.stack(images_0),
                    'x1': tf.stack(images_1)
                }, tf.stack(images_mid)

        # Create dataset from generator
        dataset = tf.data.Dataset.from_generator(
            generator,
            output_signature=(
                {
                    'time': tf.TensorSpec(shape=(None, 1), dtype=tf.float32),
                    'x0': tf.TensorSpec(shape=(None, *self.image_size, 3), dtype=tf.float32),
                    'x1': tf.TensorSpec(shape=(None, *self.image_size, 3), dtype=tf.float32)
                },
                tf.TensorSpec(shape=(None, *self.image_size, 3), dtype=tf.float32)
            )
        )
        
        return dataset.prefetch(tf.data.AUTOTUNE)

class FILMFineTuner:
    def __init__(self,
                 base_model_url: str = "https://tfhub.dev/google/film/1",
                 learning_rate: float = 1e-4):
        """
        Initialize FILM model fine-tuner
        
        Args:
            base_model_url: URL for the base FILM model
            learning_rate: Learning rate for training
        """
        self.model = hub.load(base_model_url)  # Properly load the model
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        
        # Define loss function (L1 loss)
        self.loss_fn = tf.keras.losses.MeanAbsoluteError()

        
    @tf.function
    def train_step(self, inputs: dict, target: tf.Tensor):
        """
        Perform one training step
        
        Args:
            inputs: Dictionary containing 'time', 'x0', and 'x1'
            target: Ground truth middle frame
            
        Returns:
            loss value, predicted frame
        """
        with tf.GradientTape() as tape:
            output_dict = self.model(inputs)
            predicted = output_dict['image']  # interpolated frame
            loss = self.loss_fn(target, predicted)

        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(
            zip(gradients, self.model.trainable_variables))
        
        return loss, predicted

def main():
    # Check if dataset directory exists
    data_dir = "/kaggle/working/SIH_UI_Geoserver/Frame_Interpolation/interpolation_dataset"
    if not os.path.exists(data_dir):
        raise FileNotFoundError("interpolation_dataset directory not found")
        
    # Training parameters
    BATCH_SIZE = 8
    IMAGE_SIZE = (256, 256)
    EPOCHS = 10

    # Initialize data generator
    data_generator = FILMDataGenerator(
        data_dir=data_dir,
        batch_size=BATCH_SIZE,
        image_size=IMAGE_SIZE
    )
    
    # Create strategy
    strategy = tf.distribute.MirroredStrategy()
    print("Number of devices: ", strategy.num_replicas_in_sync)
    
    with strategy.scope():
        # Initialize model
        trainer = FILMFineTuner()

        # Training loop
        for epoch in range(EPOCHS):
            print(f"\nEpoch {epoch + 1}/{EPOCHS}")
            
            # Create new dataset for this epoch
            dataset = data_generator.create_dataset()
            dist_dataset = strategy.experimental_distribute_dataset(dataset)
            
            epoch_loss = tf.keras.metrics.Mean()
            
            @tf.function
            def distributed_train_step(dist_inputs, dist_target):
                per_replica_losses, _ = strategy.run(
                    trainer.train_step, 
                    args=(dist_inputs, dist_target)
                )
                return strategy.reduce(tf.distribute.ReduceOp.MEAN, 
                                     per_replica_losses, 
                                     axis=None)
            
            # Train for exactly one epoch
            for step, (inputs, target) in enumerate(dist_dataset):
                if step >= data_generator.steps_per_epoch:
                    break
                    
                loss = distributed_train_step(inputs, target)
                epoch_loss.update_state(loss)
                
                print(f"Step {step + 1}/{data_generator.steps_per_epoch} - "
                      f"Loss: {epoch_loss.result():.4f}")
                
                # Optional checkpoint saving
                if (step + 1) % 5 == 0:  # Adjust frequency as needed
                    checkpoint_dir = f"checkpoints/epoch_{epoch}_step{step}"
                    os.makedirs(checkpoint_dir, exist_ok=True)
                    # Implement saving logic here if desired

if __name__ == "__main__":
    main()

Found 43 sample directories
Steps per epoch: 6
Number of devices:  2

Epoch 1/10
Step 1/6 - Loss: 0.1276
Step 2/6 - Loss: 0.1155
Step 3/6 - Loss: 0.1168
Step 4/6 - Loss: 0.1171
Step 5/6 - Loss: 0.1168
Step 6/6 - Loss: 0.1082

Epoch 2/10
Step 1/6 - Loss: 0.1102
Step 2/6 - Loss: 0.1128
Step 3/6 - Loss: 0.1095
Step 4/6 - Loss: 0.1087
Step 5/6 - Loss: 0.1092
Step 6/6 - Loss: 0.1063

Epoch 3/10
Step 1/6 - Loss: 0.1060
Step 2/6 - Loss: 0.1038
Step 3/6 - Loss: 0.1000
Step 4/6 - Loss: 0.1053
Step 5/6 - Loss: 0.1089
Step 6/6 - Loss: 0.1042

Epoch 4/10
Step 1/6 - Loss: 0.1211
Step 2/6 - Loss: 0.1079
Step 3/6 - Loss: 0.1085
Step 4/6 - Loss: 0.1099
Step 5/6 - Loss: 0.1059
Step 6/6 - Loss: 0.1090

Epoch 5/10
Step 1/6 - Loss: 0.0968
Step 2/6 - Loss: 0.1038
Step 3/6 - Loss: 0.1076
Step 4/6 - Loss: 0.1052
Step 5/6 - Loss: 0.1076
Step 6/6 - Loss: 0.1061

Epoch 6/10
Step 1/6 - Loss: 0.1035
Step 2/6 - Loss: 0.1012
Step 3/6 - Loss: 0.1018
Step 4/6 - Loss: 0.1071
Step 5/6 - Loss: 0.1039
Step 6/6 - Loss: 0.

In [17]:
# Ensure 'trainer' is defined and 'trainer.model' contains your trained model.
final_model_save_path = 'Models/final_model'
trainer.model.save(final_model_save_path)  # Save the Keras model
print(f"Final model saved at {final_model_save_path}")



NameError: name 'trainer' is not defined

SAVE for EACH EPOCH

In [ ]:
class FILMFineTuner:
    def __init__(self,
                 base_model_url: str = "https://tfhub.dev/google/film/1",
                 learning_rate: float = 1e-4):
        """
        Initialize FILM model fine-tuner
        
        Args:
            base_model_url: URL for the base FILM model
            learning_rate: Learning rate for training
        """
        self.model = hub.load(base_model_url)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        self.loss_fn = tf.keras.losses.MeanAbsoluteError()
        
        # Initialize checkpoint manager
        self.checkpoint = tf.train.Checkpoint(
            optimizer=self.optimizer,
            model=self.model,
            epoch=tf.Variable(0)
        )
        
    def save_model(self, save_dir: str, epoch: int = None):
        """
        Save the model weights and optimizer state
        
        Args:
            save_dir: Directory to save the model
            epoch: Current epoch number (optional)
        """
        if epoch is not None:
            self.checkpoint.epoch.assign(epoch)
            
        # Create save directory if it doesn't exist
        os.makedirs(save_dir, exist_ok=True)
        
        # Save the model weights
        weights_path = os.path.join(save_dir, 'model_weights')
        self.model.save_weights(weights_path)
        
        # Save optimizer state and epoch
        checkpoint_path = os.path.join(save_dir, 'checkpoint')
        self.checkpoint.save(checkpoint_path)
        
        print(f"Model saved successfully to {save_dir}")
        
    def load_model(self, save_dir: str):
        """
        Load the model weights and optimizer state
        
        Args:
            save_dir: Directory containing the saved model
        """
        # Load model weights
        weights_path = os.path.join(save_dir, 'model_weights')
        self.model.load_weights(weights_path)
        
        # Load optimizer state and epoch
        checkpoint_path = os.path.join(save_dir, 'checkpoint')
        self.checkpoint.restore(tf.train.latest_checkpoint(save_dir))
        
        print(f"Model loaded successfully from {save_dir}")
        return self.checkpoint.epoch.numpy()  # Return the loaded epoch number

def main():
    # ... (previous code remains the same until the training loop)
    
    # Training parameters
    BATCH_SIZE = 8
    IMAGE_SIZE = (256, 256)
    EPOCHS = 10
    SAVE_DIR = "saved_model"  # Directory to save the model
    
    with strategy.scope():
        # Initialize model
        trainer = FILMFineTuner()
        
        # Training loop
        for epoch in range(EPOCHS):
            print(f"\nEpoch {epoch + 1}/{EPOCHS}")
            
            # Create new dataset for this epoch
            dataset = data_generator.create_dataset()
            dist_dataset = strategy.experimental_distribute_dataset(dataset)
            
            epoch_loss = tf.keras.metrics.Mean()
            
            # ... (training steps remain the same)
            
            # Save model at the end of each epoch
            if (epoch + 1) % 1 == 0:  # Save every epoch (adjust frequency as needed)
                epoch_save_dir = os.path.join(SAVE_DIR, f'epoch_{epoch + 1}')
                trainer.save_model(epoch_save_dir, epoch + 1)
                
                # Save a complete model at the final epoch
                if epoch + 1 == EPOCHS:
                    final_save_dir = os.path.join(SAVE_DIR, 'final_model')
                    trainer.save_model(final_save_dir)

# To load and use the saved model later:
def load_and_use_model(save_dir: str, image1_path: str, image2_path: str, time: float = 0.5):
    """
    Load the saved model and use it for frame interpolation
    
    Args:
        save_dir: Directory containing the saved model
        image1_path: Path to first input image
        image2_path: Path to second input image
        time: Time step for interpolation (between 0 and 1)
    
    Returns:
        Interpolated frame
    """
    # Initialize model
    trainer = FILMFineTuner()
    
    # Load saved weights
    epoch = trainer.load_model(save_dir)
    print(f"Loaded model from epoch {epoch}")
    
    # Preprocess images
    def load_image(path):
        image = tf.io.read_file(path)
        image = tf.io.decode_image(image, channels=3)
        image = tf.cast(image, tf.float32) / 255.0
        image = tf.image.resize(image, (256, 256))
        return image
    
    img1 = load_image(image1_path)
    img2 = load_image(image2_path)
    
    # Prepare input
    inputs = {
        'time': tf.convert_to_tensor([[time]], dtype=tf.float32),
        'x0': tf.expand_dims(img1, 0),
        'x1': tf.expand_dims(img2, 0)
    }
    
    # Generate interpolated frame
    output = trainer.model(inputs)
    interpolated_frame = output['image'][0]  # Remove batch dimension
    
    return interpolated_frame

In [20]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
from typing import Generator, Tuple, List
from glob import glob

class FILMDataGenerator:
    def __init__(self, 
                 data_dir: str,
                 batch_size: int = 8,
                 image_size: Tuple[int, int] = (256, 256)):
        """
        Initialize the data generator for FILM model training
        
        Args:
            data_dir: Root directory containing sample folders
            batch_size: Number of samples per batch
            image_size: Target size for the images (height, width)
        """
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.image_size = image_size
        
        # Get all sample directories
        self.sample_dirs = [d for d in glob(os.path.join(data_dir, "sample_*")) 
                            if os.path.isdir(d)]
        print(f"Found {len(self.sample_dirs)} sample directories")
        
        # Calculate steps per epoch
        self.steps_per_epoch = len(self.sample_dirs) // self.batch_size
        if len(self.sample_dirs) % self.batch_size != 0:
            self.steps_per_epoch += 1
        print(f"Steps per epoch: {self.steps_per_epoch}")

    def load_and_preprocess_image(self, image_path: str) -> tf.Tensor:
        """Load and preprocess a single image"""
        # Read image file
        image = tf.io.read_file(image_path)
        # Decode image
        image = tf.io.decode_image(image, channels=3)
        # Convert to float32 and normalize to [0, 1]
        image = tf.cast(image, tf.float32) / 255.0
        # Resize image
        image = tf.image.resize(image, self.image_size)
        return image

    def create_dataset(self) -> tf.data.Dataset:
        """Create a TensorFlow dataset for training"""
        def generator():
            # Shuffle the sample directories
            indices = np.arange(len(self.sample_dirs))
            np.random.shuffle(indices)
            
            # Process in batches
            for i in range(0, len(indices), self.batch_size):
                batch_indices = indices[i:i + self.batch_size]
                batch_dirs = [self.sample_dirs[idx] for idx in batch_indices]
                
                times = []
                images_0 = []
                images_1 = []
                images_mid = []
                
                for dir_path in batch_dirs:
                    # Load the three images
                    img0 = self.load_and_preprocess_image(
                        os.path.join(dir_path, "image.png"))
                    img1 = self.load_and_preprocess_image(
                        os.path.join(dir_path, "image1.png"))
                    img2 = self.load_and_preprocess_image(
                        os.path.join(dir_path, "image2.png"))
                    
                    times.append([0.5])
                    images_0.append(img0)
                    images_1.append(img2)
                    images_mid.append(img1)
                
                # Stack into batches
                yield {
                    'time': tf.convert_to_tensor(times, dtype=tf.float32),
                    'x0': tf.stack(images_0),
                    'x1': tf.stack(images_1)
                }, tf.stack(images_mid)

        # Create dataset from generator
        dataset = tf.data.Dataset.from_generator(
            generator,
            output_signature=(
                {
                    'time': tf.TensorSpec(shape=(None, 1), dtype=tf.float32),
                    'x0': tf.TensorSpec(shape=(None, *self.image_size, 3), dtype=tf.float32),
                    'x1': tf.TensorSpec(shape=(None, *self.image_size, 3), dtype=tf.float32)
                },
                tf.TensorSpec(shape=(None, *self.image_size, 3), dtype=tf.float32)
            )
        )
        
        return dataset.prefetch(tf.data.AUTOTUNE)

class FILMFineTuner:
    def __init__(self,
                 base_model_url: str = "https://tfhub.dev/google/film/1",
                 learning_rate: float = 1e-4,
                 checkpoint_dir: str = "model_checkpoints"):
        """
        Initialize FILM model fine-tuner
        
        Args:
            base_model_url: URL for the base FILM model
            learning_rate: Learning rate for training
            checkpoint_dir: Directory to save model checkpoints
        """
        self.model = hub.load(base_model_url)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        self.loss_fn = tf.keras.losses.MeanAbsoluteError()
        self.checkpoint_dir = checkpoint_dir
        
        # Create checkpoint manager
        self.checkpoint = tf.train.Checkpoint(
            optimizer=self.optimizer,
            model=self.model
        )
        self.manager = tf.train.CheckpointManager(
            self.checkpoint,
            self.checkpoint_dir,
            max_to_keep=5  # Keep last 5 checkpoints
        )
        
    def save_checkpoint(self, epoch):
        """Save model checkpoint"""
        path = self.manager.save(checkpoint_number=epoch)
        return path
        
    def restore_latest_checkpoint(self):
        """Restore the latest checkpoint if it exists"""
        if self.manager.latest_checkpoint:
            self.checkpoint.restore(self.manager.latest_checkpoint)
            print(f"Restored from checkpoint: {self.manager.latest_checkpoint}")
            return True
        return False

    @tf.function
    def train_step(self, inputs: dict, target: tf.Tensor):
        """Train step implementation remains the same"""
        with tf.GradientTape() as tape:
            output_dict = self.model(inputs)
            predicted = output_dict['image']
            loss = self.loss_fn(target, predicted)

        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(
            zip(gradients, self.model.trainable_variables))
        
        return loss, predicted

def main():
    # Check if dataset directory exists
    data_dir = "/kaggle/working/SIH_UI_Geoserver/Frame_Interpolation/interpolation_dataset"
    if not os.path.exists(data_dir):
        raise FileNotFoundError("interpolation_dataset directory not found")
        
    # Training parameters
    BATCH_SIZE = 8
    IMAGE_SIZE = (256, 256)
    EPOCHS = 20  # Changed to 20 epochs
    CHECKPOINT_DIR = "film_model_checkpoints"
    
    # Create checkpoint directory
    os.makedirs(CHECKPOINT_DIR, exist_ok=True)

    # Initialize data generator
    data_generator = FILMDataGenerator(
        data_dir=data_dir,
        batch_size=BATCH_SIZE,
        image_size=IMAGE_SIZE
    )
    
    # Create strategy
    strategy = tf.distribute.MirroredStrategy()
    print("Number of devices: ", strategy.num_replicas_in_sync)
    
    with strategy.scope():
        # Initialize model with checkpoint directory
        trainer = FILMFineTuner(checkpoint_dir=CHECKPOINT_DIR)
        
        # Attempt to restore from checkpoint
        start_epoch = 0
        if trainer.restore_latest_checkpoint():
            # Extract the epoch number from checkpoint path
            checkpoint_path = trainer.manager.latest_checkpoint
            if checkpoint_path:
                start_epoch = int(checkpoint_path.split('ckpt-')[-1]) + 1
                print(f"Starting from epoch {start_epoch}")

        # Training loop
        for epoch in range(start_epoch, EPOCHS):
            print(f"\nEpoch {epoch + 1}/{EPOCHS}")
            
            # Create new dataset for this epoch
            dataset = data_generator.create_dataset()
            dist_dataset = strategy.experimental_distribute_dataset(dataset)
            
            epoch_loss = tf.keras.metrics.Mean()
            
            @tf.function
            def distributed_train_step(dist_inputs, dist_target):
                per_replica_losses, _ = strategy.run(
                    trainer.train_step, 
                    args=(dist_inputs, dist_target)
                )
                return strategy.reduce(tf.distribute.ReduceOp.MEAN, 
                                     per_replica_losses, 
                                     axis=None)
            
            # Train for one epoch
            for step, (inputs, target) in enumerate(dist_dataset):
                if step >= data_generator.steps_per_epoch:
                    break
                    
                loss = distributed_train_step(inputs, target)
                epoch_loss.update_state(loss)
                
                if (step + 1) % 10 == 0:  # Print every 10 steps
                    print(f"Step {step + 1}/{data_generator.steps_per_epoch} - "
                          f"Loss: {epoch_loss.result():.4f}")
            
            # Save checkpoint after each epoch
            checkpoint_path = trainer.save_checkpoint(epoch)
            print(f"\nEpoch {epoch + 1} completed. "
                  f"Average loss: {epoch_loss.result():.4f}")
            print(f"Checkpoint saved: {checkpoint_path}")
            
            # Save training metrics
            with open(os.path.join(CHECKPOINT_DIR, 'training_log.txt'), 'a') as f:
                f.write(f"Epoch {epoch + 1}, Loss: {epoch_loss.result():.4f}\n")

if __name__ == "__main__":
    main()

Found 43 sample directories
Steps per epoch: 6
Number of devices:  2

Epoch 1/20

Epoch 1 completed. Average loss: 0.1134
Checkpoint saved: film_model_checkpoints/ckpt-0

Epoch 2/20

Epoch 2 completed. Average loss: 0.1085
Checkpoint saved: film_model_checkpoints/ckpt-1

Epoch 3/20

Epoch 3 completed. Average loss: 0.1066
Checkpoint saved: film_model_checkpoints/ckpt-2

Epoch 4/20

Epoch 4 completed. Average loss: 0.1023
Checkpoint saved: film_model_checkpoints/ckpt-3

Epoch 5/20

Epoch 5 completed. Average loss: 0.1090
Checkpoint saved: film_model_checkpoints/ckpt-4

Epoch 6/20

Epoch 6 completed. Average loss: 0.1033
Checkpoint saved: film_model_checkpoints/ckpt-5

Epoch 7/20

Epoch 7 completed. Average loss: 0.1053
Checkpoint saved: film_model_checkpoints/ckpt-6

Epoch 8/20

Epoch 8 completed. Average loss: 0.1034
Checkpoint saved: film_model_checkpoints/ckpt-7

Epoch 9/20

Epoch 9 completed. Average loss: 0.1061
Checkpoint saved: film_model_checkpoints/ckpt-8

Epoch 10/20

Epoch 10

In [25]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
class FILMInferencer:
    def __init__(self, 
                 model_url: str = "https://tfhub.dev/google/film/1",
                 checkpoint_dir: str = "film_model_checkpoints"):
        """
        Initialize FILM model inferencer
        
        Args:
            model_url: URL for the FILM model
            checkpoint_dir: Directory containing saved checkpoints
        """
        self.model = hub.load(model_url)
        self.checkpoint_dir = checkpoint_dir
        self.checkpoint = tf.train.Checkpoint(model=self.model)
        self.manager = tf.train.CheckpointManager(self.checkpoint, checkpoint_dir, max_to_keep=5)

        # Restore the latest checkpoint
        if self.manager.latest_checkpoint:
            self.checkpoint.restore(self.manager.latest_checkpoint).expect_partial()
            print(f"Restored model from checkpoint: {self.manager.latest_checkpoint}")
        else:
            print("No checkpoint found. Using the pre-trained model from TensorFlow Hub.")

    def load_and_preprocess_image(self, image_path: str, image_size: tuple) -> tf.Tensor:
        """
        Load and preprocess an image for inference
        
        Args:
            image_path: Path to the image
            image_size: Target image size (height, width)
        
        Returns:
            Preprocessed image tensor
        """
        # Load and decode the image
        image = tf.io.read_file(image_path)
        image = tf.io.decode_image(image, channels=3, dtype=tf.float32)
        # Resize the image to the target size
        image = tf.image.resize(image, image_size)
        return image

    def predict_interpolated_frame(self, image0: str, image1: str, image_size: tuple = (256, 256)) -> np.ndarray:
        """
        Predict the interpolated frame between two images
        
        Args:
            image0: Path to the first input image
            image1: Path to the second input image
            image_size: Target image size (height, width)
        
        Returns:
            Interpolated frame as a NumPy array
        """
        # Load and preprocess images
        img0 = self.load_and_preprocess_image(image0, image_size)
        img1 = self.load_and_preprocess_image(image1, image_size)
        
        # Add batch dimension
        img0 = tf.expand_dims(img0, axis=0)
        img1 = tf.expand_dims(img1, axis=0)
        
        # Time tensor for interpolation (always 0.5 for midpoint frame)
        time = tf.convert_to_tensor([[0.5]], dtype=tf.float32)
        
        # Perform inference
        inputs = {'time': time, 'x0': img0, 'x1': img1}
        output_dict = self.model(inputs)
        interpolated_frame = output_dict['image'][0].numpy()
        
        # Convert to uint8 and denormalize for visualization
        interpolated_frame = (interpolated_frame * 255).astype(np.uint8)
        return interpolated_frame

# Function to visualize the images
def visualize_images(image0_path, image1_path, interpolated_frame, output_path="visualized_output.png"):
    """
    Save the visualization of two input images and the interpolated frame.

    Args:
        image0_path: Path to the first input image
        image1_path: Path to the second input image
        interpolated_frame: Interpolated frame as a NumPy array
        output_path: Path to save the visualized output
    """
    # Load input images
    image0 = cv2.imread(image0_path)
    image1 = cv2.imread(image1_path)

    # Resize input images to match the interpolated frame size
    height, width, _ = interpolated_frame.shape
    image0 = cv2.resize(image0, (width, height))
    image1 = cv2.resize(image1, (width, height))

    # Convert BGR to RGB for consistency
    image0 = cv2.cvtColor(image0, cv2.COLOR_BGR2RGB)
    image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)

    # Stack the images horizontally
    stacked_images = np.hstack((image0, interpolated_frame, image1))

    # Convert RGB to BGR for saving
    stacked_images_bgr = cv2.cvtColor(stacked_images, cv2.COLOR_RGB2BGR)

    # Save the visualized output
    cv2.imwrite(output_path, stacked_images_bgr)
    print(f"Visualization saved to: {output_path}")



def main():
    # Paths to the input images
    image0_path = "/kaggle/working/SIH_UI_Geoserver/Frame_Interpolation/interpolation_dataset/sample_1/image.png"
    image1_path = "/kaggle/working/SIH_UI_Geoserver/Frame_Interpolation/interpolation_dataset/sample_1/image2.png"
    
    # Check if files exist
    if not os.path.exists(image0_path) or not os.path.exists(image1_path):
        raise FileNotFoundError("Input image files not found")
    
    # Initialize FILM inferencer
    inferencer = FILMInferencer()
    
    # Predict the interpolated frame
    interpolated_frame = inferencer.predict_interpolated_frame(image0_path, image1_path)
    print("Interpolated frame prediction completed")
    
    # Save and visualize the result
    output_path = "interpolated_frame.png"
    cv2.imwrite(output_path, cv2.cvtColor(interpolated_frame, cv2.COLOR_RGB2BGR))
    print(f"Interpolated frame saved to: {output_path}")
    visualize_images(image0_path, image1_path, interpolated_frame, output_path="visualized_output.png")
    
    # Visualize
    # visualize_images(image0_path, image1_path, interpolated_frame)

if __name__ == "__main__":
    main()
    


Restored model from checkpoint: film_model_checkpoints/ckpt-19
Interpolated frame prediction completed
Interpolated frame saved to: interpolated_frame.png
Visualization saved to: visualized_output.png
